In [1]:
from citylearn import  CityLearn, building_loader, auto_size
from energy_models import HeatPump, EnergyStorage, Building
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
np.random.seed(3)

STATE VARIABLES
s1: outdoor temperature in Celcius degrees. This state is the same for every building for each time step.
s2: hour of day (from 1 to 24). This state is the same for every building for each time step.
s3: state of the charge of the energy storage device. From 0 (empty of cooling energy) to 1 (full of cooling energy). This state is the different for every building and depends on the actions taken by every agent (one agent per building, unless a centralized RL agent is designed to control all the buildings).

ACTION VARIABLE
a: increase (+) or decrease (-) of the amount of cooling energy stored in the energy storage device. From -0.5 (try to decrease the cooling energy stored in the storage device by an amount equivalent to 0.5 times its maximum capacity) to 0.5 (try to increase the cooling energy stored in the storage device by an amount equivalent to 0.5 times its maximum capacity). In order to decrease the energy stored in the device, the energy must be released into the building. Therefore, s3 may not decrease by the same amount than the action taken if the demand for cooling energy of the building is lower than the amount of energy the RL agent is trying to release from the storage device.

REWARD
r: -cost of electricity. See reward_function.py, which contains a function that wraps the rewards obtained from the environment. The reward function can be modified by the user in order to minimize the cost function of the environment. There may exist alternative functions different than the cost function sqrt(sum(e^2)) that have the same minima than sqrt(sum(e^2)) (i.e. penalizing the RL agent with the maximum value of electricity consumption of each day may lead to the flattening of the curve of electricity consumption too).

COST FUNCTION
env.cost(): sqrt(sum(e^2)). Where 'e' is the sum of the  electricity consumption of all the buildings in a given time-step, and sum(e^2) is the sum of the squares of 'e' over the whole simulation period. The objetive of the agent(s) must be to minimize this cost.

Any amount of cooling demand of the building that isn't satisfied by the energy storage device is automatically supplied by the heat pump directly to the building. The heat pump is the device that consumes electricity from the grid (and that has a direct impact on the rewards). The heat pump is more efficient (has a higher COP) if the outdoor air temperature is lower (s2), and less efficient (lower COP) when the outdoor temperature is higher (typically during the day time). On the other hand, the demand for cooling in the building is higher during the daytime and lower at night. COP = cooling_energy_generated/electricity_consumed. COP > 1.

SINGLE-BUILDING
For a single building, the optimal policy consists on storing cooling energy during the night (when the cooling demand of the building is low and the COP of the heat pump is higher), and releasing the stored cooling energy into the building during the day (high demand for cooling and low COP).
MULTIPLE-BUILDINGS
If multiple buildings are controlled independently of each other and with no coordination, they will all tend to consume more electricity simultaneously during the same hours at night (when the COPs are highest), raising the price for electricity that they all pay at this time and therefore the electricity cost won't be completely minimized.

SINGLE AND MULTI-AGENT RL: Proposed challenges
1- Implement an independent RL agent for every building (this has already been done in this example) and try to minimize the scores in the minimum number of episodes for multiple buildings running simultaneously. The algorithm should be properly calibrated to maximize its likelyhood of converging to a good policy (the current example does not converge 100% of the times it is run).
2- Coordinate multiple decentralized RL agents or a single centralized agent to control all the buildings. The agents could share certain information with each other (i.e. s3), while other variables (i.e. s1 and s2) are aleady common for all the agents. The agents could decide which actions to take sequentially and share this information whith other agents so they can decide what actions they will take. Pay especial attention to whether the environment (as seen by every agent) follows the Markov property or not, and how the states should be defined accordingly such that it is as Markovian as possible.

Unmark only one building ID for SINGLE AGENT environment, unmark multiple building IDs to simulate MULTI-AGENT environment.
This main file originally contains the implementation of a DDPG RL agent to control a single building/agent.

In [2]:
#Use only one building for SINGLE AGENT environment, unmark multiple building IDs to simulate MULTI-AGENT environment. In the multi-agent environment
#the reward of each agent depend partially on the actions of the other agents or buildings (see reward_function.py)
building_ids = [8]#, 5, 9, 16, 21, 26, 33, 36, 49, 59]

In [3]:
'''
Building the RL environment with heating and cooling loads and weather files
CityLearn
    Weather file
    Buildings
        File with heating and cooling demands
        CoolingDevices (HeatPump)
        CoolingStorages (EnergyStorage)
'''

data_folder = Path("data/")

demand_file = data_folder / "AustinResidential_TH.csv"
weather_file = data_folder / 'Austin_Airp_TX-hour.csv'

heat_pump, heat_tank, cooling_tank = {}, {}, {}

#Ref: Assessment of energy efficiency in electric storage water heaters (2008 Energy and Buildings)
loss_factor = 0.19/24
buildings = []
for uid in building_ids:
    heat_pump[uid] = HeatPump(nominal_power = 9e12, eta_tech = 0.22, t_target_heating = 45, t_target_cooling = 10)
    heat_tank[uid] = EnergyStorage(capacity = 9e12, loss_coeff = loss_factor)
    cooling_tank[uid] = EnergyStorage(capacity = 9e12, loss_coeff = loss_factor)
    buildings.append(Building(uid, heating_storage = heat_tank[uid], cooling_storage = cooling_tank[uid], heating_device = heat_pump[uid], cooling_device = heat_pump[uid]))
    buildings[-1].state_space(np.array([24.0, 40.0, 1.001]), np.array([1.0, 17.0, -0.001]))
    buildings[-1].action_space(np.array([0.2]), np.array([-0.2]))
    
building_loader(demand_file, weather_file, buildings)  
auto_size(buildings, t_target_heating = 45, t_target_cooling = 10)

env = CityLearn(demand_file, weather_file, buildings = buildings, time_resolution = 1, simulation_period = (3500,6000))

In [4]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

In [5]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [7]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)
        

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
            nn.Hardtanh(min_val=-0.1,max_val=0.1)
        )
        self.log_std = nn.Parameter(1/(1+np.exp(torch.ones(1, num_outputs) * std*(-1))))
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [8]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

In [9]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1]  - values[step]
        gae = delta + gamma * tau  * gae
        returns.insert(0, gae + values[step])
    return returns



In [10]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage):
    batch_size = states.size(0)
    for _ in range(batch_size // mini_batch_size):
        rand_ids = np.random.randint(0, batch_size, mini_batch_size)
        yield states[rand_ids, :], actions[rand_ids, :], log_probs[rand_ids, :], returns[rand_ids, :], advantage[rand_ids, :]
        
        

def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2):
    for _ in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            dist, value = model(state)
            entropy = dist.entropy().mean()
            new_log_probs = dist.log_prob(action)

            ratio = (new_log_probs - old_log_probs).exp()
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()
            critic_loss = (return_ - value).pow(2).mean()

            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()



In [11]:
observations_space, actions_space = [],[]
for building in buildings:
    observations_space.append(building.observation_spaces)
    actions_space.append(building.action_spaces)
num_inputs = observations_space[-1].shape[0]
num_outputs = actions_space[-1].shape[0]

#Hyper params:
hidden_size      = 32 #32,64
lr               = 0.001 #0.01,0.001,0.1
num_steps        = 20
mini_batch_size  = 24
ppo_epochs       = 4
threshold_reward = -20000

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [12]:
max_frames = 30000
frame_idx  = 0
test_rewards = []

In [14]:
envs=env

early_stop = False

k = 0
cost, cum_reward = {}, {}

from reward_function import reward_function

episodes = 30
from collections import defaultdict
state_action=defaultdict()
for e in range(episodes):
    state = envs.reset()
    
    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0
    done = False
    i=0
    while not done:
        
        l=state
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)

        action = dist.sample()
        
        action_=np.clip(action.cpu().numpy(),-0.2,0.2)
        next_state, reward, done, _ = envs.step(action_)
        state_action[int(l[0][0])]=action_
        reward=reward_function(reward)
        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        states.append(state)
        
        actions.append(action)
        
        state = next_state
        i=i+1
        

    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values
    
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)
    cost[e] = env.cost()
    print(cost[e])

214.63794822970362


KeyboardInterrupt: 